<a href="https://colab.research.google.com/github/csisyoon-hui/Data-Analysis-with-Open-Source/blob/main/%EC%98%A4%ED%94%88%EC%86%8C%EC%8A%A4_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EB%B6%84%EC%84%9D_4%EA%B0%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# 오픈소스 기반 데이터 분석 4강 - 데이터 수집


## 4-1 CSV 파일 읽기

In [6]:
import pandas as pd

## data.csv 파일 읽기
df = pd.read_csv('data.csv', encoding='utf-8', sep=',', header=0,
                 index_col=None, skiprows=None, nrows=None)
print(df)

           날짜    체중  골격근량  체지방량
0  2025.02.06  64.7  30.0  11.1
1  2025.02.04  64.0  29.3  11.6


## 4-2 JSON 파일 읽기



In [13]:
import json
import pandas as pd

## data.json 파일 출력
with open('data.json', mode='r', encoding='utf-8') as f:
    data = json.load(f)
print(data)
## data.json 파일 DataFrame 읽기
df = pd.read_json('data.json', orient='records', encoding='utf-8')
print(df)

{'매출데이터': [{'월': '2025-01', '매출액': 1000000, '비용': 700000, '이익': 300000}, {'월': '2025-02', '매출액': 1200000, '비용': 800000, '이익': 400000}, {'월': '2025-03', '매출액': 1500000, '비용': 900000, '이익': 600000}]}
                                               매출데이터
0  {'월': '2025-01', '매출액': 1000000, '비용': 700000,...
1  {'월': '2025-02', '매출액': 1200000, '비용': 800000,...
2  {'월': '2025-03', '매출액': 1500000, '비용': 900000,...


## 4-3 텍스트 파일 읽기 및 데이터 추출

In [15]:
import re

## 파일(callcenter20250301.log) 오픈 및 읽기
with open('callcenter20250301.log', 'r', encoding='utf-8') as f:
    content = f.read()
## 주민등록번호 패턴 생성
pattern = re.compile(r'(\d{6})-(\d{7})')
## 주민등록번호 마스킹
masked_content = pattern.sub(r'\1-*******', content)
## 마스킹된 파일(callcenter20250301_masked.log) 오픈 및 쓰기
with open('callcenter20250301_masked.log', mode='w') as f:
    f.write(masked_content)

print("주민등록번호 마스킹 완료. 'callcenter20250301_masked.log.txt' 파일로 저장되었습니다.")

주민등록번호 마스킹 완료. 'callcenter20250301_masked.log.txt' 파일로 저장되었습니다.


## 4-4 Open-Meteo의 무료 날씨 API를 통한 특정 지역 온도 조회

In [16]:
import requests
import json

url = "https://api.open-meteo.com/v1/forecast?=&=&current=temperature_2m"
params = {
    "latitude": "37.58638333",
    "longitude": "127.0203333",
    "current": "temperature_2m"
}

try:
    ## URL 및 파라미터 전송
    response = requests.get(url, params=params)
    response.raise_for_status()

    ## JSON 데이터 읽기
    data = response.json()
    print("API 응답:", data)
    print("서울시 종로구의 현재 온도는 : {0}{1} 입니다.".format(data['current']['temperature_2m'], data['current_units']['temperature_2m']))

except requests.exceptions.RequestException as e:
    print(f"API 호출 실패: {e}")
except json.JSONDecodeError as e:
    print(f"JSON 파싱 실패: {e}")

API 응답: {'latitude': 37.6, 'longitude': 127.0, 'generationtime_ms': 0.03886222839355469, 'utc_offset_seconds': 0, 'timezone': 'GMT', 'timezone_abbreviation': 'GMT', 'elevation': 29.0, 'current_units': {'time': 'iso8601', 'interval': 'seconds', 'temperature_2m': '°C'}, 'current': {'time': '2025-10-22T06:15', 'interval': 900, 'temperature_2m': 18.2}}
서울시 종로구의 현재 온도는 : 18.2°C 입니다.


## 4-5 Selenium과 lxml을 이용한 웹 스크래핑

In [17]:
!curl -o google-chrome-stable_current_amd64.deb https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!apt install ./google-chrome-stable_current_amd64.deb -y
!pip install selenium webdriver_manager

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  114M  100  114M    0     0   328M      0 --:--:-- --:--:-- --:--:--  328M
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'google-chrome-stable' instead of './google-chrome-stable_current_amd64.deb'
The following additional packages will be installed:
  libvulkan1 mesa-vulkan-drivers
The following NEW packages will be installed:
  google-chrome-stable libvulkan1 mesa-vulkan-drivers
0 upgraded, 3 newly installed, 0 to remove and 38 not upgraded.
Need to get 10.9 MB/131 MB of archives.
After this operation, 447 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libvulkan1 amd64 1.3.204.1-2 [128 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 mesa-vulkan-drivers amd64 23.2.1-1ubuntu3.1~22.04.3 [10.7

In [19]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from lxml import html
import time

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')               # 브라우저 창 없이 실행
chrome_options.add_argument('--no-sandbox')             # 보안모드 비활성화 (Colab 필수)
chrome_options.add_argument('--disable-dev-shm-usage')  # 메모리 부족 방지 (Colab 필수)
chrome_options.add_argument('--window-size=1920x1080')  # 창 크기 설정(가상)
chrome_options.add_argument('--disable-gpu')            # GPU 가속 비활성화 (일부 환경 안정성)
chrome_options.binary_location = "/usr/bin/google-chrome-stable"  # Colab용 크롬 경로 지정

## 드라이버 실행
driver = webdriver.Chrome(options=chrome_options)

## 사이트 접속
url = 'https://professor.knou.ac.kr/jaehwachung/index.do'
driver.get(url)
## 사이트 접속 대기
time.sleep(2)

## 페이지 제목 출력
page_source = driver.page_source
tree = html.fromstring(page_source)

title_text = tree.xpath('//title/text()')
print(title_text)
## 드라이버 종료
driver.quit()

['컴퓨터과학과 정재화 교수']



# 실습 시나리오

## 공공데이터 포털 가입 및 데이터 신청

- [https://www.data.go.kr](https://www.data.go.kr)
- 한국환경공단 에어코리아 대기오염정보 데이터 신청

In [ ]:
import requests

## 데이터 수집 url 및 api key 설정

url = 'http://apis.data.go.kr/B552584/ArpltnInforInqireSvc/getCtprvnRltmMesureDnsty'
api_key = '~~~~'
params = {
    'serviceKey': api_key,
    'returnType': 'json',
    'numOfRows': '100',
    'pageNo': '1',
    'sidoName': '서울',
    'ver': '1.0'
}

## 데이터 수집
response = requests.get(url, params=params)
## 호출 성공/실패 출력
print(response.json())


In [20]:
import requests
import time
import json
import os
from datetime import datetime
from tqdm import tqdm
import pandas as pd

API_KEY = "4b666641786373693732524e756442"  # <-- 본인의 API 키로 교체하세요
BASE_URL = "http://openapi.seoul.go.kr:8088/{key}/json/energyUseDataSummaryInfo/1/5/{year}/{month}"

START_YEAR = 2015
END_YEAR = 2024

OUT_DIR = "energy_data"
os.makedirs(OUT_DIR, exist_ok=True)

results = []
errors = []

# helper to format month as two digits
def fmt_month(m):
    return f"{m:02d}"

# 요청 반복 (연도, 월)
for year in range(START_YEAR, END_YEAR + 1):
    for month in range(1, 13):
        # 마지막 해의 마지막 달까지만 수집
        if year == END_YEAR and month > 12:
            break

        url = BASE_URL.format(key=API_KEY, year=year, month=fmt_month(month))
        # API 호출 (간단한 재시도 로직 포함)
        max_retries = 3
        for attempt in range(1, max_retries + 1):
            try:
                resp = requests.get(url, timeout=15)
                status = resp.status_code

                # 캡처용 출력(콘솔에 표시)
                print(f"[{datetime.now().isoformat()}] 요청: {year}-{fmt_month(month)}  URL: {url}")
                print(f"  상태코드: {status}")

                if status == 200:
                    # JSON 파싱
                    data = resp.json()
                    # API의 응답 구조에 따라 실제 데이터가 다른 키 안에 있을 수 있음.
                    # 예시: data['energyUseDataSummaryInfo']['row'] 등
                    # 안전하게 추출 시도:
                    payload = None
                    if isinstance(data, dict):
                        # 흔한 구조 예시 검사
                        if 'energyUseDataSummaryInfo' in data:
                            block = data['energyUseDataSummaryInfo']
                            if isinstance(block, dict) and 'row' in block:
                                payload = block['row']
                        # fallback: 전체 dict에서 첫 object에 row가 있는지 확인
                        if payload is None:
                            for v in data.values():
                                if isinstance(v, dict) and 'row' in v:
                                    payload = v['row']
                                    break
                    # payload가 None이면 전체 JSON을 기록
                    entry = {
                        "year": year,
                        "month": fmt_month(month),
                        "url": url,
                        "status_code": status,
                        "raw_json": data,
                        "parsed_rows": payload
                    }
                    results.append(entry)

                    # 월별 원본 JSON 저장 (사후 확인/캡처용)
                    out_path = os.path.join(OUT_DIR, f"{year}_{fmt_month(month)}.json")
                    with open(out_path, "w", encoding="utf-8") as f:
                        json.dump(entry, f, ensure_ascii=False, indent=2)

                    # 성공하면 재시도 루프 탈출
                    break
                else:
                    # 에러 응답일 경우 응답 본문 로그
                    text = resp.text[:500]
                    print(f"  응답본문(최대500자): {text}")
                    # 서버 오류(5xx)라면 재시도, 클라이언트 오류(4xx)는 재시도 안함
                    if 500 <= status < 600:
                        print(f"  서버 오류({status}) — 재시도 {attempt}/{max_retries} ...")
                        time.sleep(1 + attempt)  # backoff
                        continue
                    else:
                        errors.append({"year": year, "month": fmt_month(month), "status": status, "text": text})
                        break

            except requests.exceptions.RequestException as e:
                print(f"  예외 발생: {e}  — 재시도 {attempt}/{max_retries}")
                time.sleep(1 + attempt)
                if attempt == max_retries:
                    errors.append({"year": year, "month": fmt_month(month), "error": str(e)})

        # API 호출 사이에 잠깐 지연을 넣어 과도한 요청을 피함
        time.sleep(0.2)

# 통합된 테이블(가능하면 parsed_rows에서 필요한 필드만 추출)
rows_for_df = []
for item in results:
    parsed = item.get("parsed_rows")
    if parsed and isinstance(parsed, list):
        for r in parsed:
            # 예시 필드명 가정: 'USE_YYYYMM', 'ENERGY_TYPE', 'USAGE' 등
            # 실제 필드명은 API 응답을 보고 조정해야 합니다.
            row = {
                "year": item["year"],
                "month": item["month"],
                "raw": r
            }
            # 일반적으로 'raw' 내부의 키들을 풀어 CSV 칼럼으로 만들려면 아래처럼 확장
            # for k,v in r.items(): row[k] = v
            rows_for_df.append(row)
    else:
        # parsed_rows가 없으면 raw_json을 기록한 한 줄
        rows_for_df.append({
            "year": item["year"],
            "month": item["month"],
            "raw": item["raw_json"]
        })

df = pd.json_normalize(rows_for_df)
csv_path = os.path.join(OUT_DIR, "energy_summary_2015_2024.csv")
df.to_csv(csv_path, index=False, encoding="utf-8-sig")

# 요약 출력(캡처용)
print("\n=== 수집 요약 ===")
print(f"총 월 요청 수: {(END_YEAR-START_YEAR+1)*12}")
print(f"성공적으로 받은 응답 수: {len(results)}")
print(f"에러/예외 건수: {len(errors)}")
print(f"저장된 CSV 파일: {csv_path}")
print(f"원본 JSON 파일들은 폴더: {OUT_DIR} 에 저장됨")

[2025-10-22T08:34:11.162968] 요청: 2015-01  URL: http://openapi.seoul.go.kr:8088/4b666641786373693732524e756442/json/energyUseDataSummaryInfo/1/5/2015/01
  상태코드: 200
[2025-10-22T08:34:11.813671] 요청: 2015-02  URL: http://openapi.seoul.go.kr:8088/4b666641786373693732524e756442/json/energyUseDataSummaryInfo/1/5/2015/02
  상태코드: 200
[2025-10-22T08:34:12.482545] 요청: 2015-03  URL: http://openapi.seoul.go.kr:8088/4b666641786373693732524e756442/json/energyUseDataSummaryInfo/1/5/2015/03
  상태코드: 200
[2025-10-22T08:34:13.124836] 요청: 2015-04  URL: http://openapi.seoul.go.kr:8088/4b666641786373693732524e756442/json/energyUseDataSummaryInfo/1/5/2015/04
  상태코드: 200
[2025-10-22T08:34:13.762143] 요청: 2015-05  URL: http://openapi.seoul.go.kr:8088/4b666641786373693732524e756442/json/energyUseDataSummaryInfo/1/5/2015/05
  상태코드: 200
[2025-10-22T08:34:14.417494] 요청: 2015-06  URL: http://openapi.seoul.go.kr:8088/4b666641786373693732524e756442/json/energyUseDataSummaryInfo/1/5/2015/06
  상태코드: 200
[2025-10-22T08:3